# Distributed Training

Deep learning models are infamous in their resources needs and not having these resources available can make Large models and / or large data slow the training time drastically, making it almost impossible to run at a reasonable time. Luckly for us, MLRun can assign GPUs and distribute the training with ease, all from the same `apply_mlrun` function we saw in the previous notebook.

In this notebook, we will train a large MNIST model by assigning GPUs and distribute the training using Horovod.

1. [Open MPI and Horovod](#section_1)
2. [Load the Projet](#section_2)
3. [Run Distributed Training](#section_3)

___
<a id="section_1"></a>

## 1. Open MPI and Horovod

MLRun is using Horovod over OpenMPI to run distributed training, we will have a brief explanation on both before we continue with loading the project and running our training function.

### 1.1. Open MPI

<img src="https://www.open-mpi.org/images/open-mpi-logo.png" alt="Open MPI logo" width="150"/>

From the official [**Open MPI**](https://www.open-mpi.org/) website: *The Open MPI Project is an open source Message Passing Interface implementation that is developed and maintained by a consortium of academic, research, and industry partners. Open MPI is therefore able to combine the expertise, technologies, and resources from all across the High Performance Computing community in order to build the best MPI library available. Open MPI offers advantages for system and software vendors, application developers and computer science researchers.*

To use Open MPI in MLRun, when creating a MLRun function all that is needed to be done is to set the `kind` parameter to `"mpijob"`. We will cover it in chapter 3.

### 1.2. Horovod

<img src="https://horovod.readthedocs.io/en/stable/_static/logo.png" alt="Horovod logo" width="150"/>

From the official [**Horovod**](https://horovod.ai/) website: *Horovod was originally developed by Uber to make distributed deep learning fast and easy to use, bringing model training time down from days and weeks to hours and minutes.*

Becuase we used `apply_mlrun` in our code, MLRun will take care of all the configuration required to distribute the training. All we need to do is to create function with `kind="mpijob"`.

___
<a id="section_2"></a>

## 2. Load the Project

We will load our project using the same function we used to create it: 

In [1]:
import mlrun

# Set our project's name:
project_name = "mnist-classifier"

# Create the project:
project = mlrun.get_or_create_project(name=project_name, context="./", user_project=True)

> 2022-08-24 21:38:23,701 [warning] Failed resolving version info. Ignoring and using defaults
> 2022-08-24 21:38:26,980 [warning] Server or client version is unstable. Assuming compatible: {'server_version': '1.1.0-rc11', 'client_version': '0.0.0+unstable'}
> 2022-08-24 21:38:27,310 [info] loaded project mnist-classifier from MLRun DB


___
<a id="section_3"></a>

## 3. Run Distributed Training

Although it may sound complicated, in MLRun a distributed training job is just another job. We will need to create an Open MPI job from our training code, configure the amount of workers and run it. That's all!

### 3.1. Create the Funnction

We will use `code_to_function` again on the model development notebook, but this time we will set `kind` to `"mpijob"`:

In [2]:
# Create the function parsing the model development code from previous notebook using 'code_to_function':
mpijob_development_function = mlrun.code_to_function(
    filename="./model_development.ipynb",  # <- We set the function to get the code from this notebook.
    name="mnist-mpijob-development",
    kind="mpijob",  # <- Notice we set kind to be an OpenMPI job.
)

# Save the function in the project:
project.set_function(mpijob_development_function)
project.save()

# Mount it:
mpijob_development_function.apply(mlrun.platforms.auto_mount())

### 3.2. Configure the Function

We will configure to train with 2 workers. In addition, we will choose what resources to use for each worker, set `use_gpu` to `True` if you wish to use GPUs, the image will be set accorindgly (hence we didn't set it at `code_to_function` using the `image` attribute). 

In [3]:
# Select whether to use GPUs:
use_gpu = True

# Set the image and GPUs if needed:
if use_gpu:
    mpijob_development_function.spec.image = "mlrun/ml-models-gpu"
    # Select the number of GPUs per replica:
    mpijob_development_function.gpus(1)
else:
    mpijob_development_function.spec.image = "mlrun/ml-models"

# Setup the number of workers for training:
mpijob_development_function.spec.replicas = 2

### 3.3. Run a Distributed Training MPIJob

Let's call `run` and see our training in action. Notice the number of steps is half as it was, as now each worker is using half of the dataset.

Checking the function in MLRun's UI, we can see in the `pods` tab the workers we assign (2 in our case). If chosen to use GPUs and your system has GPUs scaled to zero, it may take time to assign them, so the `pods` tab is a great visual way to see the state of your workers:

<img src="./mlrun_ui_results.png" alt="MLRun UI pods tab screenshot" width="1200"/>

In [4]:
# Run the distributed training job:
distributed_training_run = mpijob_development_function.run(
    handler="train",
    name="distributed-training",
    params={
        "conv_blocks": 4,
        "dense_blocks": 4,
        "learning_rate": 1e-3,
        "batch_size": 128,
        "epochs": 20,
    },
)

> 2022-08-24 21:38:40,653 [info] starting run distributed-training uid=226cccdc9b944106ab2d0e781b08b0c5 DB=http://mlrun-api:8080
> 2022-08-24 21:41:43,021 [info] MpiJob status unknown or failed, check pods: {'distributed-training-76c19f91-launcher': 'Pending', 'distributed-training-76c19f91-worker-0': 'Pending', 'distributed-training-76c19f91-worker-1': 'Pending'}
+ + POD_NAME=distributed-training-76c19f91-worker-0POD_NAME=distributed-training-76c19f91-worker-1

+ + shiftshift

+ + /opt/kube/kubectl/opt/kube/kubectl exec exec distributed-training-76c19f91-worker-1 distributed-training-76c19f91-worker-0 -- -- /bin/sh /bin/sh -c -c        PATH=/usr/local/bin:$PATH ; export PATH ; LD_LIBRARY_PATH=/usr/local/lib:$LD_LIBRARY_PATH ; export LD_LIBRARY_PATH ; DYLD_LIBRARY_PATH=/usr/local/lib:$DYLD_LIBRARY_PATH ; export DYLD_LIBRARY_PATH ;   /usr/local/bin/orted -mca ess "env" -mca ess_base_jobid "3290497024" -mca ess_base_vpid 1 -mca ess_base_num_procs "3" -mca orte_node_regex "distributed-tra

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
mnist-classifier-guyl,...1b08b0c5,0,Aug 24 21:52:30,completed,distributed-training,v3io_user=guylkind=mpijobowner=guylmlrun/client_version=0.0.0+unstablemlrun/job=distributed-training-76c19f91host=distributed-training-76c19f91-worker-0,,conv_blocks=4dense_blocks=4learning_rate=0.001batch_size=128epochs=20,conv_blocks=4dense_blocks=4learning_rate=0.001batch_size=128epochs=20lr=0.0020000000949949026training_loss=0.11069107055664062training_accuracy=0.96875validation_loss=0.10850751142394036validation_accuracy=0.9878333071444897,training_loss.htmltraining_accuracy.htmlvalidation_loss.htmlvalidation_accuracy.htmlloss_summary.htmlaccuracy_summary.htmllr_values.htmlmodel


> 2022-08-24 22:06:25,452 [info] run executed, status=completed


___
Lastly, the [**next chapter**](./model_serving.ipynb) will be around serving our model in a realtime serverless function with a cool Jupyter application, see you there!